In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from autogluon.tabular import TabularPredictor

random.seed(0)
np.random.seed(0)


df_train = pd.read_excel("train.xlsx")
df_test = pd.read_excel("test.xlsx")

In [ ]:
df_train

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,26169,20230310-7492-177993190,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,26170,20230625-16563-206126520,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Активный,Номер «Студия»,3,4
26171,26171,20220624-7492-137587082,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,26172,20220427-7491-125459150,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Отмена,Номер «Стандарт»,2,3


In [ ]:
def prepare_df(df):
    df = df.copy()
    df = df.drop(["Unnamed: 0", "№ брони"], axis=1)
    df["Дата бронирования"] = pd.to_datetime(df["Дата бронирования"])
    df["Заезд"] = pd.to_datetime(df["Заезд"])
    df["Выезд"] = pd.to_datetime(df["Выезд"])
    return df


def create_date_features(df, prefix):
    df = df.copy()
    df[prefix + "_month"] = df[prefix].dt.month.astype("int8")
    df[prefix + "_day_of_month"] = df[prefix].dt.day.astype("int8")
    df[prefix + "_day_of_year"] = df[prefix].dt.dayofyear.astype("int16")
    df[prefix + "_week_of_month"] = (
        df[prefix].apply(lambda d: (d.day - 1) // 7 + 1)
    ).astype("int8")
    df[prefix + "_week_of_year"] = (df[prefix].dt.isocalendar().week).astype("int8")
    df[prefix + "_day_of_week"] = (df[prefix].dt.dayofweek + 1).astype("int8")
    df[prefix + "_year"] = df[prefix].dt.year.astype("int32")
    df[prefix + "_is_wknd"] = (df[prefix].dt.weekday // 4).astype("int8")
    df[prefix + "_season"] = np.where(df[prefix + "_month"].isin([12, 1, 2]), 0, 1)
    df[prefix + "_season"] = np.where(
        df[prefix + "_month"].isin([6, 7, 8]), 2, df[prefix + "_season"]
    )
    df[prefix + "_season"] = pd.Series(
        np.where(df[prefix + "_month"].isin([9, 10, 11]), 3, df[prefix + "_season"])
    ).astype("int8")
    return df


def create_diff_features(df, prefix1, prefix2):
    df = df.copy()
    df[prefix1 + "_" + prefix2 + "_diff_in_days"] = (df[prefix1] - df[prefix2]).dt.days
    df[prefix1 + "_" + prefix2 + "_diff_in_weeks"] = (
        df[prefix1 + "_" + prefix2 + "_diff_in_days"] / 7
    )
    df[prefix1 + "_" + prefix2 + "_diff_in_hours"] = (
        df[prefix1] - df[prefix2]
    ).dt.total_seconds() / 3600
    return df


def create_payment_method_features(df):
    df = df.copy()
    df["SberPay"] = df["Способ оплаты"].apply(lambda x: int("SberPay" in x))
    df["Yandex Pay"] = df["Способ оплаты"].apply(lambda x: int("Yandex Pay" in x))
    df["МИР"] = df["Способ оплаты"].apply(lambda x: int("МИР" in x))
    df["ComfortBooking"] = df["Способ оплаты"].apply(lambda x: int("ComfortBooking" in x))
    df["TravelLine Pro"] = df["Способ оплаты"].apply(lambda x: int("TravelLine Pro" in x))
    df["Банк Россия"] = df["Способ оплаты"].apply(lambda x: int("Банк Россия" in x))
    df["Внешняя система оплаты"] = df["Способ оплаты"].apply(
        lambda x: int("Внешняя система оплаты" in x)
    )
    df["Банковская карта"] = df["Способ оплаты"].apply(
        lambda x: int(
            "Банковская карта" in x
            or "Банк. карта".lower() in x.lower()
            or "банковской картой".lower() in x.lower()
        )
    )
    df["Оплата наличными"] = df["Способ оплаты"].apply(
        lambda x: int("Оплата наличными" in x)
    )
    df["С предоплатой"] = df["Способ оплаты"].apply(lambda x: int("С предоплатой" in x))
    df["СБП"] = df["Способ оплаты"].apply(lambda x: int("Система быстрых платежей" in x))

    df["Отложенная электронная оплата"] = df["Способ оплаты"].apply(
        lambda x: int("Отложенная электронная оплата" in x)
    )

    df["Гарантия банковской картой"] = df["Способ оплаты"].apply(
        lambda x: int("Гарантия банковской картой" in x)
    )

    df["При заселении"] = df["Способ оплаты"].apply(lambda x: int("При заселении" in x))

    return df


def clear_source_column(text):
    # good
    mapping = [
        "Официальный сайт",
        "Бронирование из экстранета",
        "Яндекс.Путешествия",
        "ostrovok",
        "booking",
        "Программа лояльности",
        "Bronevik",
        "OneTwoTrip",
    ]

    for map_ in mapping:
        if map_.lower() in text.lower():
            return map_
    return "other"


def clear_category(text):
    # good
    if "\n" in text:
        text = text.split("\n")
        text = text[0]
    text = text.strip("1. ")
    return text


def add_some_exciting_stuff(col):
    if col.month in [2, 3, 4] and col.year == 2022:
        return 1
    if col.month == 9 and col.year == 2022:
        return 1
    if col.month == 7 and col.year == 2023:
        return 1
    return 0

In [ ]:
def preprocess_df(df):
    df = df.copy()
    df = prepare_df(df)
    df = create_date_features(df, "Дата бронирования")
    df = create_date_features(df, "Заезд")
    df = create_date_features(df, "Выезд")
    df = create_diff_features(df, "Заезд", "Дата бронирования")
    df = create_diff_features(df, "Выезд", "Заезд")

    df = create_payment_method_features(df)

    # df["Источник"] = df["Источник"].apply(clear_source_column)

    df["Категория multiple selection"] = df["Категория номера"].apply(
        lambda x: int("\n" in x)
    )

    # df["Категория номера"] = df["Категория номера"].apply(clear_category)

    # other features

    df["Стоимость за ночь"] = df["Стоимость"] / df["Ночей"]
    df["Внесена предоплата binary"] = df.apply(
        lambda x: int(x["Внесена предоплата"] != 0), axis=1
    )

    df["Предоплата умноженная на время до прибытия"] = (
        df["Внесена предоплата"] * df["Заезд_Дата бронирования_diff_in_days"]
    )

    df["Процент предоплата от стоимости"] = df["Внесена предоплата"] / df["Стоимость"]
    df["Гостей на номер"] = df["Гостей"] / df["Номеров"]

    df["Стоимость на гостя"] = df["Стоимость"] / df["Гостей"]
    df["Стоимость за номер"] = df["Стоимость"] / df["Номеров"]
    df["Стоимость за ночь 1 номер"] = df["Стоимость"] / df["Ночей"] / df["Номеров"]
    df["Стоимость за ночь 1 гостя"] = df["Стоимость"] / df["Ночей"] / df["Гостей"]
    df["Гостей на номер"] = df["Гостей"] / df["Номеров"]
    df["Бронирование утром"] = df["Дата бронирования"].apply(
        lambda x: (
            1
            if x.time() >= pd.Timestamp("05:00:00").time()
            and x.time() < pd.Timestamp("11:00:00").time()
            else 0
        )
    )
    df["Бронирование вечером"] = df["Дата бронирования"].apply(
        lambda x: (
            1
            if x.time() >= pd.Timestamp("17:00:00").time()
            and x.time() < pd.Timestamp("23:00:00").time()
            else 0
        )
    )
    df["Бронирование ночью"] = df["Дата бронирования"].apply(
        lambda x: (
            1
            if x.time() >= pd.Timestamp("23:00:00").time()
            or x.time() < pd.Timestamp("05:00:00").time()
            else 0
        )
    )

    df["Заезд_top_cancels"] = df["Заезд"].apply(add_some_exciting_stuff)
    df["Выезд_top_cancels"] = df["Выезд"].apply(add_some_exciting_stuff)
    df["Дата бронирования"] = df["Дата бронирования"].apply(add_some_exciting_stuff)

    return df

In [ ]:
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)

In [ ]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,...,Гостей на номер,Стоимость на гостя,Стоимость за номер,Стоимость за ночь 1 номер,Стоимость за ночь 1 гостя,Бронирование утром,Бронирование вечером,Бронирование ночью,Заезд_top_cancels,Выезд_top_cancels
0,1,25700.0,0,Внешняя система оплаты,0,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,...,2.0,12850.0,25700.0,8566.666667,4283.333333,0,1,0,0,0
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,0,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,...,2.0,12400.0,24800.0,12400.000000,6200.000000,0,0,0,0,0
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),0,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,...,2.0,12900.0,25800.0,12900.000000,6450.000000,0,1,0,0,0
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),0,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),...,1.0,10500.0,10500.0,10500.000000,10500.000000,0,0,0,0,0
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,0,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,...,4.0,7172.5,28690.0,14345.000000,3586.250000,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),0,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,...,2.0,9120.0,18240.0,9120.000000,4560.000000,0,1,0,0,0
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),0,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,...,3.0,23200.0,69600.0,23200.000000,7733.333333,0,1,0,0,0
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),0,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,...,2.0,27800.0,55600.0,13900.000000,6950.000000,0,1,0,0,0
26172,1,6300.0,0,Гарантия банковской картой,1,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,...,2.0,3150.0,6300.0,6300.000000,3150.000000,1,0,0,1,1


In [ ]:
# drops
df_train = df_train.drop(["Дата бронирования", "Заезд", "Выезд"], axis=1)
df_test = df_test.drop(["Дата бронирования", "Заезд", "Выезд"], axis=1)

In [ ]:
df_train["target"] = df_train["Дата отмены"].apply(lambda x: int(pd.notna(x)))
df_train = df_train.drop(["Дата отмены", "Статус брони"], axis=1)

In [ ]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Источник,Категория номера,Гостей,Гостиница,Дата бронирования_month,...,Стоимость на гостя,Стоимость за номер,Стоимость за ночь 1 номер,Стоимость за ночь 1 гостя,Бронирование утром,Бронирование вечером,Бронирование ночью,Заезд_top_cancels,Выезд_top_cancels,target
0,1,25700.0,0,Внешняя система оплаты,3,Яндекс.Путешествия,Номер «Стандарт»,2,1,4,...,12850.0,25700.0,8566.666667,4283.333333,0,1,0,0,0,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2,Официальный сайт,Номер «Стандарт»,2,1,6,...,12400.0,24800.0,12400.000000,6200.000000,0,0,0,0,0,0
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2,Официальный сайт,Номер «Студия»,2,4,11,...,12900.0,25800.0,12900.000000,6450.000000,0,1,0,0,0,0
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1,Bronevik.com(new),Номер «Стандарт»,1,3,9,...,10500.0,10500.0,10500.000000,10500.000000,0,0,0,0,0,0
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2,Официальный сайт,Номер «Люкс»,4,1,5,...,7172.5,28690.0,14345.000000,3586.250000,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2,Официальный сайт,Номер «Стандарт»,2,2,1,...,9120.0,18240.0,9120.000000,4560.000000,0,1,0,0,0,0
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),3,Официальный сайт,Номер «Студия»,3,4,6,...,23200.0,69600.0,23200.000000,7733.333333,0,1,0,0,0,0
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),4,Официальный сайт,Номер «Стандарт»,2,2,5,...,27800.0,55600.0,13900.000000,6950.000000,0,1,0,0,0,0
26172,1,6300.0,0,Гарантия банковской картой,1,booking.com,Номер «Стандарт»,2,3,2,...,3150.0,6300.0,6300.000000,3150.000000,1,0,0,1,1,1


In [ ]:
predictor = TabularPredictor(
    label="target", problem_type="binary", eval_metric="roc_auc"
).fit(df_train, presets="best_quality", time_limit=1400)

No path specified. Models will be saved in: "AutogluonModels/ag-20240922_121633"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Wed Aug  7 16:19:28 UTC 2024
CPU Count:          16
Memory Avail:       7.19 GB / 15.49 GB (46.4%)
Disk Space Avail:   185.83 GB / 468.09 GB (39.7%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

In [ ]:
predictions = predictor.predict_proba(df_test)

In [ ]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.865790,roc_auc,5.685200,308.627782,0.003163,1.834201,2,True,36
1,LightGBM_r96_BAG_L1,0.862067,roc_auc,1.446372,6.597293,1.446372,6.597293,1,True,19
2,LightGBM_r131_BAG_L1,0.861847,roc_auc,0.538486,5.256319,0.538486,5.256319,1,True,16
3,LightGBMXT_BAG_L1,0.861833,roc_auc,0.180819,2.543201,0.180819,2.543201,1,True,3
4,CatBoost_r137_BAG_L1,0.861568,roc_auc,0.126688,35.676327,0.126688,35.676327,1,True,23
5,LightGBM_BAG_L1,0.861451,roc_auc,0.124340,2.257870,0.124340,2.257870,1,True,4
6,CatBoost_r9_BAG_L1,0.861234,roc_auc,0.143850,38.254063,0.143850,38.254063,1,True,18
7,XGBoost_r89_BAG_L1,0.861223,roc_auc,0.198707,3.688209,0.198707,3.688209,1,True,29
8,CatBoost_r177_BAG_L1,0.860899,roc_auc,0.115091,24.345269,0.115091,24.345269,1,True,14
9,XGBoost_r194_BAG_L1,0.860746,roc_auc,0.182009,4.169248,0.182009,4.169248,1,True,35


In [ ]:
predictions

,0,1
0,0.957711,0.042289
1,0.917223,0.082777
2,0.873908,0.126092
3,0.881671,0.118329
4,0.932652,0.067348
...,...,...
11213,0.881835,0.118165
11214,0.820844,0.179156
11215,0.981109,0.018891
11216,0.949142,0.050858


: 

In [ ]:
predictions[1].to_csv("autogluon_on_processed_data.csv", index=False, header=False)